# Good Books Dataset Preparation

Link: https://www.kaggle.com/zygmunt/goodbooks-10k

### Import Library

In [198]:
import pandas as pd #data wrangling
import re #regex  -> https://regex101.com/

### Import Dataset

In [199]:
# import dataset
books = pd.read_csv('dataset/books.csv')
book_tags = pd.read_csv('dataset/book_tags.csv')
tags = pd.read_csv('dataset/tags.csv')

In [200]:
print('Books dataset contains:',books.shape[0],'rows and',books.shape[1],'columns.')
print('Book Tags dataset contains:',book_tags.shape[0],'rows and',book_tags.shape[1],'columns.')
print('Tags dataset contains:',tags.shape[0],'rows and',tags.shape[1],'columns.')

Books dataset contains: 10000 rows and 23 columns.
Book Tags dataset contains: 999912 rows and 3 columns.
Tags dataset contains: 34252 rows and 2 columns.


### Data Cleansing

In [210]:
#Copy Tags dataset
tags_copy = tags.copy()

In [211]:
#lowercase
tags_copy = tags_copy.applymap(lambda s: s.lower() if type(s) == str else s)

In [212]:
# remove special characters using regex
for i in range(len(tags_copy['tag_name'])):
    tags_copy['tag_name'][i] = re.sub(r"[^a-z]", " ", tags_copy['tag_name'][i])

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [213]:
# trim string    
tags_copy = tags_copy.applymap(lambda s: s.strip() if type(s) == str else s)    

In [214]:
#filter empty string
tags_copy = tags_copy[tags_copy.tag_name != '']

#reset index
tags_copy.reset_index()
tags_copy = tags_copy[['tag_id','tag_name']]

In [215]:
print('Tags dataset after cleansing using regex:',tags_copy.shape[0],'rows,',tags_copy.shape[1],'columns.')

Tags dataset after cleansing using regex: 33105 rows, 2 columns.


#### Filtering unnecessary tags

In [216]:
#tags that will be keep that indicates genre
included_tags = ["art", "biography", "business", 
                 "chick lit", "childrens", "christian",
                 "classics", "comics", "contemporary", 
                 "cookbooks", "crime", "ebooks", 
                 "fantasy", "fiction", "gay and lesbian", 
                 "graphic novels", "historical fiction", "history", 
                 "horror", "humor and comedy", "manga", 
                 "memoir", "music", "mystery", 
                 "nonfiction", "paranormal", "philosophy", 
                 "poetry", "psychology", "religion", 
                 "romance", "science", "science fiction", 
                 "self help", "suspense", "spirituality", 
                 "sports", "thriller", "travel", 
                 "young adult"]

In [217]:
#temporary list
new_list_ids = []
new_list_names = []

In [218]:
#filter process using for loop
for i in range(len(tags_copy['tag_name'])):
    for i_t in included_tags:
        if i_t in tags_copy['tag_name'].iloc[i]:
            tag_ids = new_list_ids.append(tags_copy['tag_id'].iloc[i])
            tag_names = new_list_names.append(i_t)

In [219]:
#after filtering process above, we convert that list to dataframe
new_tags_copy = pd.DataFrame({"tag_id" :new_list_ids, "tag_name" : new_list_names})
print('New tags dataset in final form contains:',new_tags_copy.shape[0],'rows,',new_tags_copy.shape[1],'columns.')

New tags dataset in final form contains: 5186 rows, 2 columns.


### Join the data

Book Tag --join-- Tag

In [230]:
#join final tag dataset to book tag dataset
new_book_tag = book_tags.merge(new_tags_copy, how='left', left_on='tag_id', right_on='tag_id')

#drop any missing joins
new_book_tag = new_book_tag[new_book_tag['tag_name'].notnull()]

print('New book tags dataset after joined with tag dataset contains:',new_book_tag.shape[0],'rows,',new_book_tag.shape[1],'columns.')

New book tags dataset after joined with tag dataset contains: 262592 rows, 4 columns.


#### Filter any books that had more than one tag(keep the highest count)

In [231]:
#drop duplicates, but keep the one which got highest count
new_book_tag = new_book_tag.sort_values(by=["goodreads_book_id", 'count'], ascending=True).drop_duplicates(subset=["goodreads_book_id"],keep='last')

#reset index
new_book_tag = new_book_tag.reset_index()
new_book_tag = new_book_tag[['goodreads_book_id','tag_id','count','tag_name']]

new_book_tag.head()

,goodreads_book_id,tag_id,count,tag_name
0,1,11305,37174,fantasy
1,2,11305,3441,fantasy
2,3,11305,47478,fantasy
3,5,11305,39330,fantasy
4,6,11305,38378,fantasy


Books --join-- New Book Tag

In [262]:
#join final tag dataset to book tag dataset
new_books = books.merge(new_book_tag, how='left', left_on='best_book_id', right_on='goodreads_book_id')

#drop any missing joins
new_books = new_books[new_books['goodreads_book_id'].notnull()]

print('New book tags dataset after joined with tag dataset contains:',new_book_tag.shape[0],'rows,',new_book_tag.shape[1],'columns.')

New book tags dataset after joined with tag dataset contains: 10000 rows, 4 columns.


In [263]:
new_books.drop(['best_book_id','goodreads_book_id','isbn','isbn13'], axis = 1, inplace=True)
new_books.head()

,id,book_id,work_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,tag_id,count,tag_name
0,1,2767052,2792775,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,33114.0,25968.0,young adult
1,2,3,4640799,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,11305.0,47478.0,fantasy
2,3,41865,3212258,226,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,...,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,33114.0,16446.0,young adult
3,4,2657,3275794,487,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,...,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,7457.0,31068.0,classics
4,5,4671,245494,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,...,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,7457.0,32061.0,classics


In [266]:
# export new dataset to csv file
new_books.to_csv('new_books.csv', index=False)
new_book_tag.to_csv('new_book_tag.csv', index=False)
new_tags_copy.to_csv('new_tags.csv', index=False)